<a href="https://colab.research.google.com/github/murffps/machine-learning-titanic/blob/main/MLBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile requirements.txt
matplotlib
pandas
sklearn 
yellowbrick
xlrd

In [ ]:
!pip install -r requirements.txt
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

In [ ]:


# source [https://www.oreilly.com/library/view/machine-learning-pocket/9781492047537/]
# (Type of ML Prediction) Classification Supervised Learning : 
# Classify whether an individual survies the Titanic ship catastrophe 
#   based on individual and trip characteristics. (transform features into a label)
# Predict if they will (did) survive based on certain characteristic
# Would they survive based on ...

# import tools and libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (ensemble, preprocessing,tree)
from sklearn.metrics import (auc, confusion_matrix, roc_auc_score, roc_curve)
from sklearn.model_selection import (train_test_split, StratifiedKFold)
from yellowbrick.classifier import (ConfusionMatrix, ROCAUC)
from yellowbrick.model_selection import (LearningCurve)
import ydata_profiling
from ydata_profiling import ProfileReport

# GATHER DATA
url = ("https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls")
df = pd.read_excel(url)
orig_df = df

# CLEAN DATA - Standardize, Outliers(check min/max), Missing Data(check for NaN)
ydata_profiling.ProfileReport(df)
df.shape
profile = ProfileReport(df, title="data set", html={'style' : {'full_width':True}})
profile.to_file(output_file="file.html")


# profile
df.describe().iloc[:, :2]

# missing data 
# conclusions: age (interpolate values), cabin, boat, body all going to be dropped -LEAKY - 
df.isnull().sum()

df.isnull().mean() * 100

# inspect some specific data
# decide on strategy to replace or deal with missing data  
# a. Use S the most common to  
# b. drop them -- One possible strategy is to drop columns with no variance or signal or perfect or very high positive or negative correlation
# c. use pandas to create dummy columns with all 0s 
df.sex.value_counts(dropna=False)

df.embarked.value_counts(dropna=False)

## CREATE FEATURES 
name = df.name
name.head(3)

df = df.drop(
    columns=[
        "name",
        "ticket",
        "home.dest",
        "boat",
        "body",
        "cabin"
    ])

df = pd.get_dummies(df)
df.columns

df = df.drop(columns="sex_male")
# OR df = pd.get_dummies(df, drop_first=True)
df.columns

y = df.survived
X = df.drop(columns="survived")

## SAMPLE DATA - pull out 30% 
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
#not working NameError: name 'model_selection' is not defined

# Impute missing values for age on training set 

In [ ]:
# View HTML file 
import IPython
IPython.display.HTML(filename='/content/file.html')